In [1]:
import os
from keras.models import * 
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
from multiprocessing import cpu_count
import tensorflow as tf

Using TensorFlow backend.


In [2]:
nb_classes = 21
nb_cpus = 8
nb_gpus = 4
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,2,3'

image_size = (299, 299)
input_shape= (299, 299, 3)

In [3]:
with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(1.0)(p_out)
    predictions = Dense(nb_classes, activation='softmax')(p_out)
    model = Model(inputs=base_model.input, outputs=predictions)
    
model = multi_gpu_model(model, gpus=nb_gpus)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_5 (

In [4]:
model.load_weights('batch6.4_007_0.0655.hdf5')

In [5]:
batch_size = 64
test_path = "/home/nvme/liyu/xcp-batch6.4/CELLS-half/valid"

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(test_path, 
                                         target_size=image_size, 
                                         shuffle=False,
                                         batch_size=batch_size)
test_img_nums = test_generator.samples
all_test_results = model.predict_generator(test_generator, 
                                           len(test_generator), 
                                           workers=nb_cpus, 
                                           use_multiprocessing=True,
                                           verbose=1)
all_labels = test_generator.classes

Found 43605 images belonging to 21 classes.
682/682 [==============================] - 132s 193ms/step


In [6]:
class_label_dict = test_generator.class_indices
print(test_generator.class_indices)
def get_key(dict_, value):
    return [k for k, v in dict_.items() if v == value]

# create class num lens dict, every dict store current class predict num
total_predictions_dict = {}
for row_class_name, label in class_label_dict.items():
    total_predictions_dict[row_class_name] = {}
    for column_class_name, label in class_label_dict.items():
        total_predictions_dict[row_class_name][column_class_name] = 0

{'LSIL_E': 11, 'ACTINO': 0, 'SCC_R': 18, 'PH': 14, 'ASCUS': 3, 'HSIL_M': 9, 'EC': 5, 'VIRUS': 20, 'HSIL_S': 10, 'AGC_B': 2, 'GEC': 7, 'HSIL_B': 8, 'AGC_A': 1, 'TRI': 19, 'FUNGI': 6, 'SC': 16, 'MC': 13, 'CC': 4, 'SCC_G': 17, 'RC': 15, 'LSIL_F': 12}


In [16]:
thresh = 0.95

for i, label in enumerate(all_labels):
    predict_index = np.argmax(all_test_results[i])
    predict_det = all_test_results[i][predict_index]
    if (predict_det > thresh):
        # get the first result
        label_class_name = get_key(class_label_dict, label)[0]
        test_class_name = get_key(class_label_dict, np.argmax(all_test_results[i]))[0]
#     print(label_class_name)
#     print(total_predictions_dict[label_class_name].keys())
        total_predictions_dict[label_class_name][test_class_name] += 1

In [17]:
print(total_predictions_dict['RC'])

{'LSIL_E': 0, 'ACTINO': 0, 'VIRUS': 0, 'ASCUS': 0, 'PH': 0, 'HSIL_M': 0, 'EC': 1, 'HSIL_S': 0, 'LSIL_F': 0, 'GEC': 11, 'AGC_B': 0, 'SCC_R': 0, 'HSIL_B': 0, 'AGC_A': 0, 'TRI': 0, 'FUNGI': 0, 'RC': 3411, 'MC': 0, 'CC': 0, 'SCC_G': 0, 'SC': 0}


In [18]:
import csv
out = open('confusion_matrix_95.csv','a', newline='')
csv_write = csv.writer(out,dialect='excel')

# write the title
line = [class_name for class_name, label in class_label_dict.items()]
line = [" "] + line + ["TOTAL"] + ["ACC"]
csv_write.writerow(line)

# write rows

true_num = 0
all_num = 0

for row_class_name, label in class_label_dict.items():
    one_class_total_predict = 0
    line = [row_class_name]
    for column_class_name, label in class_label_dict.items():
        one_class_total_predict += total_predictions_dict[row_class_name][column_class_name]
    
    for column_class_name, label in class_label_dict.items():
        one_class_cur_predict = total_predictions_dict[row_class_name][column_class_name]
        # acc
        #acc = round((one_class_cur_predict / one_class_total_predict), 4)
        #line.append(str(acc))
        # num
        line.append(one_class_cur_predict)
    print(one_class_total_predict)
    print(str(one_class_total_predict))
    line.append(str(one_class_total_predict))
    line.append(round((total_predictions_dict[row_class_name][row_class_name] / one_class_total_predict), 4))       
    print(line)
    csv_write.writerow(line)
    
    true_num += total_predictions_dict[row_class_name][row_class_name]
    all_num += one_class_total_predict
    
csv_write.writerow(["ALL_ACC"] + [round((true_num / all_num), 4)])
    
out.close()

5045
5045
['LSIL_E', 4993, 0, 0, 1, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 33, '5045', 0.9897]
11259
11259
['ACTINO', 0, 11259, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '11259', 1.0]
5305
5305
['SCC_R', 0, 0, 5296, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, '5305', 0.9983]
6002
6002
['PH', 0, 0, 0, 5931, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 22, 23, 0, 0, 0, '6002', 0.9882]
4707
4707
['ASCUS', 16, 0, 0, 12, 4560, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 117, '4707', 0.9688]
5587
5587
['HSIL_M', 0, 0, 0, 0, 0, 5531, 0, 1, 0, 15, 5, 29, 0, 0, 0, 0, 0, 0, 6, 0, 0, '5587', 0.99]
1403
1403
['EC', 0, 0, 0, 0, 0, 20, 1316, 0, 0, 34, 18, 0, 0, 0, 0, 0, 0, 0, 0, 15, 0, '1403', 0.938]
5537
5537
['VIRUS', 0, 0, 0, 0, 0, 0, 0, 5470, 50, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 1, '5537', 0.9879]
22202
22202
['HSIL_S', 0, 0, 10, 0, 2, 6, 0, 11, 21971, 0, 2, 0, 0, 0, 0, 105, 0, 0, 95, 0, 0, '22202', 0.9896]
1402
1402
['AGC_B', 0, 0, 0, 0, 0, 10, 29, 0, 0, 1346, 13, 4